In [1]:
# FLASK_APP=dash_app/app.py flask run
import pandas as pd
import numpy as np
from flask import Flask, jsonify, render_template
import sqlite3
import re

In [2]:
pwd

'/Users/x22/Desktop/MyGTdata/MyHW/d3js'

* SQL DATA

In [315]:
conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
cur = conn.cursor()

coordinates_query = '''
    SELECT * from country_coordinates
'''
cur.execute(coordinates_query)
coordinates_rows = cur.fetchall()
coordinates_rows[0]

('AD', 42.546245, 1.6015540000000001, 'Andorra')

In [316]:
coordinates = []

for row in coordinates_rows:
    tempDict = {}
    tempDict["code"] = row[0]
    tempDict["location"] = [row[1], row[2]]
    tempDict["latitude"] = row[1]
    tempDict["longitude"] = row[2]
    tempDict["name"] = row[3]
    coordinates.append(tempDict)
coordinates[0]

{'code': 'AD',
 'location': [42.546245, 1.6015540000000001],
 'latitude': 42.546245,
 'longitude': 1.6015540000000001,
 'name': 'Andorra'}

In [317]:
country_query = '''
    SELECT Country, count(country) FROM jso11k
    GROUP BY Country
    ORDER BY COUNT(Country) DESC
'''

cur.execute(country_query)
country_rows = cur.fetchall()
country_rows[0]


('United States', 2597)

In [318]:
countries = []
for row in country_rows:
    tempDict = {}
    tempDict["country"] = (row[0])
    tempDict["respondentCount"] = int(row[1])
    # tempDict[row[0]] = int(row[1])
    countries.append(tempDict)
countries[:2]

[{'country': 'United States', 'respondentCount': 2597},
 {'country': 'India', 'respondentCount': 1077}]

In [319]:
for i in range(len(countries)):
    if countries[i]['country'] == 'Russian Federation':
        countries[i]['country'] = 'Russia'
    if countries[i]['country'] == 'Czech Republic':
        countries[i]['country'] = 'Czech Republic'
    if countries[i]['country'] == 'Viet Nam':
        countries[i]['country'] = 'Vietnam'
    if countries[i]['country'] == 'Venezuela, Bolivarian Republic of...':
        countries[i]['country'] = 'Venezuela'
    if countries[i]['country'] == 'Republic of Korea':
        countries[i]['country'] = 'North Korea'
    if countries[i]['country'] == 'Syrian Arab Republic':
        countries[i]['country'] = 'Syria'
    if countries[i]['country'] == "Lao People's Democratic Republic":
        countries[i]['country'] = 'Laos'
    if countries[i]['country'] == 'Sao Tome and Principe':
        countries[i]['country'] = 'São Tomé and Príncipe'
    if countries[i]['country'] == 'The former Yugoslav Republic of Macedonia':
        countries[i]['country'] = 'Macedonia [FYROM]'
    if countries[i]['country'] == 'Republic of Moldova':
        countries[i]['country'] = 'Moldova'
    if countries[i]['country'] == 'United Republic of Tanzania':
        countries[i]['country'] = 'Tanzania'
    if countries[i]['country'] == 'Democratic Republic of the Congo':
        countries[i]['country'] = 'Congo [DRC]'
        

In [320]:
for i in range(len(coordinates)):
    for j in range(len(countries)):
        if coordinates[i]['name'] in countries[j].values():
            countries[j]['code'] = str(coordinates[i]['code'])
            countries[j]['latitude'] = float(coordinates[i]['latitude'])
            countries[j]['longitude'] = float(coordinates[i]['longitude'])
            countries[j]['location'] = (coordinates[i]['location'])

In [321]:
countries[0]

{'country': 'United States',
 'respondentCount': 2597,
 'code': 'US',
 'latitude': 37.09024,
 'longitude': -95.712891,
 'location': [37.09024, -95.712891]}

In [322]:
stats_query = '''
    SELECT * FROM country_stats
'''

cur.execute(stats_query)
stats_rows = cur.fetchall()
stats_rows[0]

('China',
 1420062022,
 '0.35 %',
 5016094,
 151,
 9388211,
 -324919.0,
 '1.6',
 '39',
 '60 %',
 '18.41 %')

In [323]:
stats = []

for row in stats_rows:
    tempDict = {}
    tempDict["country"] = row[0]
    tempDict["population"] = row[1]
    stats.append(tempDict)
stats[0]

{'country': 'China', 'population': 1420062022}

In [324]:
for i in range(len(stats)):
    for j in range(len(countries)):
        if stats[i]['country'] in countries[j].values():
            countries[j]['population'] = stats[i]['population']
            countries[j]['devsPerMill'] = countries[j]['respondentCount']/(stats[j]['population']/1000000)

In [329]:
len(countries)
(round(3.14651515, 2))

3.15

In [326]:
for country in countries:
    if 'population' not in country.keys():
        print(country)
#     print(country['population'])
    

{'country': 'Czech Republic', 'respondentCount': 82, 'code': 'CZ', 'latitude': 49.817492, 'longitude': 15.472961999999999, 'location': [49.817492, 15.472961999999999]}
{'country': 'Hong Kong (S.A.R.)', 'respondentCount': 26}
{'country': 'Other Country (Not Listed Above)', 'respondentCount': 14}
{'country': 'Macedonia [FYROM]', 'respondentCount': 5, 'code': 'MK', 'latitude': 41.608635, 'longitude': 21.745275, 'location': [41.608635, 21.745275]}
{'country': 'Cape Verde', 'respondentCount': 1, 'code': 'CV', 'latitude': 16.002082, 'longitude': -24.013197, 'location': [16.002082, -24.013197]}
{'country': 'Congo [DRC]', 'respondentCount': 1, 'code': 'CD', 'latitude': -4.038333000000001, 'longitude': 21.758664000000003, 'location': [-4.038333000000001, 21.758664000000003]}
{'country': 'São Tomé and Príncipe', 'respondentCount': 1, 'code': 'ST', 'latitude': 0.18636, 'longitude': 6.613081, 'location': [0.18636, 6.613081]}
{'country': None, 'respondentCount': 0}


In [162]:
for i in range(len(countries)):
    try:
    # populationScale = countries[i]['pupulation']/1000000
        countries[i]['devsPerMill'] = int(countries[i]['respondentCount'])/int(countries[i]['population'])
#         print(countries[i])
    except:
#         continue
        print("error occurred")

error occurred
error occurred
error occurred
error occurred
error occurred
error occurred
error occurred
error occurred
error occurred
error occurred
error occurred
error occurred
error occurred
error occurred
error occurred


In [257]:
countries_no_population = []
no_population = []
for country in countries:
    if 'population' not in country.keys():
#         no_population.append(country)
#         print(country['population'])
        countries_no_population.append(country)
#         print(country)
#     print(country['population'])
countries_no_population

[{'country': 'Russian Federation', 'respondentCount': 214},
 {'country': 'Czech Republic',
  'respondentCount': 82,
  'code': 'CZ',
  'latitude': 49.817492,
  'longitude': 15.472961999999999,
  'location': [49.817492, 15.472961999999999]},
 {'country': 'Viet Nam', 'respondentCount': 32},
 {'country': 'Hong Kong (S.A.R.)', 'respondentCount': 26},
 {'country': 'Other Country (Not Listed Above)', 'respondentCount': 14},
 {'country': 'Venezuela, Bolivarian Republic of...', 'respondentCount': 11},
 {'country': 'Republic of Korea', 'respondentCount': 5},
 {'country': 'The former Yugoslav Republic of Macedonia',
  'respondentCount': 5},
 {'country': 'Republic of Moldova', 'respondentCount': 4},
 {'country': 'Syrian Arab Republic', 'respondentCount': 4},
 {'country': 'United Republic of Tanzania', 'respondentCount': 3},
 {'country': 'Cape Verde',
  'respondentCount': 1,
  'code': 'CV',
  'latitude': 16.002082,
  'longitude': -24.013197,
  'location': [16.002082, -24.013197]},
 {'country': 'Dem

In [261]:
for i in range(len(countries)):
    if countries[i]['country'] == 'Russian Federation':
        countries[i]['country'] = 'Russia'
    if countries[i]['country'] == 'Czech Republic':
        countries[i]['country'] = 'Czech Republic'
    if countries[i]['country'] == 'Viet Nam':
        countries[i]['country'] = 'Vietnam'
    if countries[i]['country'] == 'Venezuela, Bolivarian Republic of...':
        countries[i]['country'] = 'Venezuela'
    if countries[i]['country'] == 'Republic of Korea':
        countries[i]['country'] = 'North Korea'
    if countries[i]['country'] == 'Syrian Arab Republic':
        countries[i]['country'] = 'Syria'
    if countries[i]['country'] == "Lao People's Democratic Republic":
        countries[i]['country'] = 'Laos'
    if countries[i]['country'] == 'Sao Tome and Principe':
        countries[i]['country'] = 'São Tomé and Príncipe'
    if countries[i]['country'] == 'The former Yugoslav Republic of Macedonia':
        countries[i]['country'] = 'Macedonia [FYROM]'
    if countries[i]['country'] == 'Republic of Moldova':
        countries[i]['country'] = 'Moldova'
    if countries[i]['country'] == 'United Republic of Tanzania':
        countries[i]['country'] = 'Tanzania'
    if countries[i]['country'] == 'Democratic Republic of the Congo':
        countries[i]['country'] = 'Congo [DRC]'
        

In [260]:
for i in range(len(countries)):
    if countries[i]['country'] == 'Russian Federation':
        print(countries[i])

In [256]:
c = [item['country'] for item in countries_with_population ]
c[0] = 'Russia'
c[1] = 

['Russia',
 'Czech Republic',
 'Viet Nam',
 'Hong Kong (S.A.R.)',
 'Other Country (Not Listed Above)',
 'Venezuela, Bolivarian Republic of...',
 'Republic of Korea',
 'The former Yugoslav Republic of Macedonia',
 'Republic of Moldova',
 'Syrian Arab Republic',
 'United Republic of Tanzania',
 'Cape Verde',
 'Democratic Republic of the Congo',
 "Lao People's Democratic Republic",
 'Sao Tome and Principe',
 None]

In [250]:
for i in range(len(countries)):
    if countries[i]['country'] == 'Russian Federation':
        print(countries[i])
        countries[i]['country'] == 'Russia'
        print(countries[i]['country'])
        

{'country': 'Russian Federation', 'respondentCount': 214}
Russian Federation


In [246]:
rus = {'country': 'Russian Federation', 'respondentCount': 214}

In [249]:
rus['country'] = 'Russia'
rus

{'country': 'Russia', 'respondentCount': 214}

In [222]:
len(countries_with_population)

0

In [233]:
countries_with_population = []
for country in countries:
    if 'population' in country.keys():
        countries_with_population.append(country)

In [235]:
len(countries_with_population)

127

In [218]:
countries_with_population = [country for country in countries if 'population' in country.keys()]

In [219]:
len(countries_with_population)

0

In [216]:
len(countries)

143

In [159]:
countries = [item for item in countries if item['country'] is not None]


In [160]:

countries = [item for item in countries if item['population'] is not None]


KeyError: 'population'

In [152]:
len(countries)

142

In [132]:
for country in countries[:10]:
    print(country['population'])

KeyError: 'population'

* SCRAPED DATA for latitude and longitude

In [77]:
type(countries[0]['population'])

int

In [33]:
country_url = "https://developers.google.com/public-data/docs/canonical/countries_csv"

In [34]:
country_tables = pd.read_html(country_url)

In [39]:
country_tables[:3]

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan


In [40]:
country_tables.to_csv("developer_survey_2019/country_coordinates.csv", index=False)

In [41]:
country_tables.to_sql(name='country_coordinates', con=conn, if_exists='replace', index=False)


In [53]:
cur.execute("select * from country_coordinates")

In [54]:
cur.fetchall()[:2]

[('AD', 42.546245, 1.6015540000000001, 'Andorra'),
 ('AE', 23.424076, 53.847818000000004, 'United Arab Emirates')]

In [208]:
countries[0]

{'country': 'United States',
 'respondentCount': 2597,
 'code': 'US',
 'latitude': 37.09024,
 'longitude': -95.712891,
 'location': [37.09024, -95.712891],
 'population': 329093110}

In [205]:
stats[:3]

[{'country': 'China', 'population': 1420062022},
 {'country': 'India', 'population': 1368737513},
 {'country': 'United States', 'population': 329093110}]

In [ ]:
stats_query = '''
    SELECT * FROM country_stats
'''

cur.execute(stats_query)
stats_rows = cur.fetchall()

stats = []

for row in stats_rows:
    print(row[1])
    tempDict = {}
    tempDict["country"] = row[0]
    tempDict["population"] = row[1]
    stats.append(tempDict)

In [201]:
stats[:5]

[{'country': 'China', 'population': 1420062022},
 {'country': 'India', 'population': 1368737513},
 {'country': 'United States', 'population': 329093110},
 {'country': 'Indonesia', 'population': 269536482},
 {'country': 'Brazil', 'population': 212392717}]

In [174]:
countries[:2]

[{'country': 'United States',
  'respondentCount': 2597,
  'code': 'US',
  'latitude': 37.09024,
  'longitude': -95.712891,
  'location': [37.09024, -95.712891]},
 {'country': 'India',
  'respondentCount': 1077,
  'code': 'IN',
  'latitude': 20.593684,
  'longitude': 78.96288,
  'location': [20.593684, 78.96288]}]

In [135]:
countries[-2:]

[{'country': 'Yemen',
  'respondentCount': 1,
  'code': 'YE',
  'latitude': 15.552726999999999,
  'longitude': 48.516388},
 {'country': 'Zambia',
  'respondentCount': 1,
  'code': 'ZM',
  'latitude': -13.133897,
  'longitude': 27.849332}]

In [108]:
len(countries)

143

In [181]:
stats_url = 'https://www.worldometers.info/world-population/population-by-country/'

In [182]:
country_stats = pd.read_html(stats_url)

In [183]:
country_stats_df = country_stats[0]

In [184]:
print(len(country_stats_df))



233


In [185]:
country_stats_df.columns


Index(['#', 'Country (or dependency)', 'Population (2019)', 'Yearly Change',
       'Net Change', 'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)',
       'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share'],
      dtype='object')

In [186]:
del country_stats_df['#']

In [187]:
country_stats_df.columns

Index(['Country (or dependency)', 'Population (2019)', 'Yearly Change',
       'Net Change', 'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)',
       'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share'],
      dtype='object')

In [188]:
country_stats_df.columns = ['Country', 'Population', 'YearlyChange', 'NetChange',\
              'Density', 'LandAreaKm2', 'MigrantsNet', 'FertRate',\
              'MedAge', 'UrbanPop%', 'WorldShare']

In [189]:
country_stats_df[:2]

,Country,Population,YearlyChange,NetChange,Density,LandAreaKm2,MigrantsNet,FertRate,MedAge,UrbanPop%,WorldShare
0,China,1420062022,0.35 %,5016094,151,9388211,-324919.0,1.6,39,60 %,18.41 %
1,India,1368737513,1.08 %,14685659,460,2973190,-490000.0,2.3,28,34 %,17.74 %


In [190]:
country_stats_df.to_csv("developer_survey_2019/worldPopulation2019.csv", index=False)

In [191]:
country_stats_df['Country'][:2]

0    China
1    India
Name: Country, dtype: object

In [192]:
len(country_stats_df)

233

In [193]:
wiki_countries = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)') 

In [209]:
wiki_countries[1].columns

Index(['Rank', 'Country or area', 'UN continentalregion[2]',
       'UN statisticalregion[2]', 'Population(1 July 2018)[3]',
       'Population(1 July 2019)[3]', 'Change'],
      dtype='object')

### create country stats sql table

In [167]:
pwd

'/Users/x22/Desktop/MyGTdata/MyHW/d3js'

In [179]:
country_stats_df.to_sql(name='country_stats', con=conn, if_exists='replace', index=False)


In [183]:
cur.execute("select * from country_stats")

In [184]:
rows = cur.fetchall()

In [186]:
# type(stats[0][1])
rows[0]

('China',
 1420062022,
 '0.35 %',
 5016094,
 151,
 9388211,
 -324919.0,
 '1.6',
 '39',
 '60 %',
 '18.41 %')

### add coordinates to count of respondents by country(from original survey)

In [ ]:
# Russian Federation Coordinates: 61.5240, 105.3188
conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
cur = conn.cursor()

coordinates_query = '''
    SELECT * from country_coordinates
'''
cur.execute(coordinates_query)
coordinates_rows = cur.fetchall()

coordinates = []

for row in coordinates_rows:
    tempDict = {}
    tempDict["code"] = row[0]
    tempDict["location"] = [row[1], row[2]]
    tempDict["latitude"] = row[1]
    tempDict["longitude"] = row[2]
    tempDict["name"] = row[3]
    coordinates.append(tempDict)
    
country_query = '''
    SELECT Country, count(country) FROM jso11k
    GROUP BY Country
    ORDER BY COUNT(Country) DESC
'''

cur.execute(country_query)
country_rows = cur.fetchall()

countries = []

for row in country_rows:
    tempDict = {}
    tempDict["country"] = (row[0])
    tempDict["respondentCount"] = int(row[1])
    # tempDict[row[0]] = int(row[1])
    countries.append(tempDict)
    
for i in range(len(coordinates)):
    for j in range(len(countries)):
        if coordinates[i]['name'] in countries[j].values():
            countries[j]['code'] = str(coordinates[i]['code'])
            countries[j]['latitude'] = float(coordinates[i]['latitude'])
            countries[j]['longitude'] = float(coordinates[i]['longitude'])
            countries[j]['location'] = (coordinates[i]['location'])

            
stats_query = '''
    SELECT * FROM country_stats
'''

cur.execute(stats_query)
stats_rows = cur.fetchall()

stats = []

for row in stats_rows:
    tempDict = {}
    tempDict["country"] = row[0]
    tempDict["population"] = row[1]
    stats.append(tempDict)
    
for i in range(len(stats)):
    for j in range(len(countries)):
        if stats[i]['country'] in countries[j].values():
            countries[j]['population'] = stats[i]['population']
            
countries = [item for item in countries if item['country'] is not None]


## Respondent Count by Gender

In [116]:
genders_query = "SELECT Gender, COUNT(Gender) FROM jso11k GROUP BY Gender"

In [117]:
cur.execute(genders_query)
rows = cur.fetchall()

In [118]:
rows

[(None, 0),
 ('Man', 9588),
 ('Man;Non-binary, genderqueer, or gender non-conforming', 23),
 ('Non-binary, genderqueer, or gender non-conforming', 70),
 ('Woman', 813),
 ('Woman;Man', 15),
 ('Woman;Man;Non-binary, genderqueer, or gender non-conforming', 8),
 ('Woman;Non-binary, genderqueer, or gender non-conforming', 20)]

In [119]:
data = []
for row in rows:
    temp_dict = {}
    temp_dict[row[0]] = row[1]
    gender_data.append(temp_dict)

# return jsonify(gender_data)

In [120]:
data

[{None: 0},
 {'Man': 9588},
 {'Man;Non-binary, genderqueer, or gender non-conforming': 23},
 {'Non-binary, genderqueer, or gender non-conforming': 70},
 {'Woman': 813},
 {'Woman;Man': 15},
 {'Woman;Man;Non-binary, genderqueer, or gender non-conforming': 8},
 {'Woman;Non-binary, genderqueer, or gender non-conforming': 20}]

### Impostor Syndrom

In [124]:
# "select count(ImpSyn) from jso88k"
cur.execute("select ImpSyn, count(ImpSyn) from jso11k group by ImpSyn")
rows = cur.fetchall()
data = []
for row in rows:
    temp_dict = {}
    temp_dict[row[0]] = row[1]
    data.append(temp_dict)
data

[{None: 0},
 {'A little above average': 3904},
 {'A little below average': 663},
 {'Average': 2128},
 {'Far above average': 1920},
 {'Far below average': 191}]

In [135]:
def get_data(query_string):
    conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
    cur = conn.cursor()
    cur.execute(query_string)
    rows = cur.fetchall()
    data = []
    for row in rows:
        temp_dict = {}
        temp_dict[row[0]] = row[1]
        data.append(temp_dict)
    return data

In [138]:
q = "select ImpSyn, count(ImpSyn) from jso11k group by ImpSyn"

In [139]:
ImpSyn = get_data(q)

In [141]:
ImpSyn

list

## Languages frequency in Men

In [39]:
conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
cur = conn.cursor()

In [90]:
queries = {
        "man": '''
            SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith)
            FROM jso11k WHERE Gender = 'Man' and LanguageWorkedWith > 0
            GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith)
        ''',
        "woman": '''
            SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith)
            FROM jso11k
            WHERE Gender = 'Woman' and LanguageWorkedWith > 0
            GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith) DESC
        ''',
        "other": '''
            SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith)
            FROM jso11k
            WHERE Gender NOT IN ('Man', 'Woman') and LanguageWorkedWith > 0
            GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith) DESC
        '''
    }

languages_by_gender = []

for key, value in queries.items():

    cur.execute(value)
    rows = cur.fetchall()

    language_freq = {}

    for row in rows:
        languages = row[0].split(';')   
        for item in languages:
            if item in language_freq:
#                 language_freq['']
                language_freq[item] += int(row[1])
            else:
                language_freq[item] = int(row[1])

    languages_by_gender.append([{'gender': key, 'languages':language_freq}])
    

# languages_by_gender = []

# for key, value in queries.items():

#     cur.execute(value)
#     rows = cur.fetchall()

#     language_freq = {'gender': '', 'language': '', 'count':0}

#     for row in rows:
#         languages = row[0].split(';')
#         for item in languages:
# #             if item in language_freq.values():
#             if item == language_freq['language']:
# #                 language_freq['gender'] = key
# #                 language_freq['language'] = item
#                 language_freq['count'] += int(row[1])
#             else:
#                 language_freq['gender'] = key
#                 language_freq['language'] = item
#                 language_freq['count'] = row[1] #int[row[1]]
# #                 language_freq[item] = int(row[1])

# #     languages_by_gender.append({"gender":key, "languages:":language_freq})
#         languages_by_gender.append(language_freq)
    

In [91]:
languages_by_gender

[[{'gender': 'man',
   'languages': {'Assembly': 648,
    'Bash/Shell/PowerShell': 3528,
    'C#': 3061,
    'Go': 816,
    'HTML/CSS': 6014,
    'JavaScript': 6419,
    'PHP': 2489,
    'Python': 3971,
    'Ruby': 749,
    'SQL': 5326,
    'TypeScript': 2060,
    'VBA': 533,
    'Swift': 586,
    'Java': 3921,
    'R': 564,
    'Scala': 383,
    'Objective-C': 430,
    'Rust': 303,
    'WebAssembly': 113,
    'Other(s):': 865,
    'C++': 2317,
    'Clojure': 139,
    'Dart': 168,
    'Elixir': 139,
    'Erlang': 86,
    'F#': 123,
    'Kotlin': 591,
    'C': 1995}}],
 [{'gender': 'woman',
   'languages': {'HTML/CSS': 530,
    'JavaScript': 517,
    'Python': 313,
    'C#': 180,
    'SQL': 386,
    'Java': 284,
    'PHP': 182,
    'TypeScript': 130,
    'Bash/Shell/PowerShell': 225,
    'R': 75,
    'Go': 38,
    'Ruby': 85,
    'C': 121,
    'C++': 144,
    'Kotlin': 30,
    'Assembly': 55,
    'Objective-C': 25,
    'Swift': 44,
    'Other(s):': 58,
    'WebAssembly': 8,
    'Rust': 

In [42]:
languages_by_gender = []


In [107]:
cur.execute('''SELECT Gender, LanguageWorkedWith, COUNT(LanguageWorkedWith)
FROM jso11k WHERE Gender = 'Man' and LanguageWorkedWith > 0
GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith)
''')

In [108]:
rows = cur.fetchall()
print(len(rows))
rows[:2]

4108


[('Man', 'Assembly;Bash/Shell/PowerShell', 1),
 ('Man',
  'Assembly;Bash/Shell/PowerShell;C#;Go;HTML/CSS;JavaScript;PHP;Python;Ruby;SQL;TypeScript;VBA',
  1)]

In [110]:
language_data = []

In [111]:
for row in rows:
    tempDict = {}
    tempDict['gender'] = row[0]
    tempDict['language'] = row[1]
    tempDict['count'] = row[2]
    language_data.append(tempDict)
#     languages = row[1].split(';')
    
#     for language in languages:
#         counter = {'language': '', 'count': 0}
#         if counter['language'] == language:
#             counter['count'] += row[2]
        
#         tempDict['gender'] = row[0]
#         tempDict['language'] = language
#         tempDict['count'] = row[2]
        
    
# country_data = []

# for row in rows:
#     tempDict = {}
#     tempDict["country"] = row[0]
#     tempDict["respondentCount"] = int(row[1])
#     # tempDict[row[0]] = int(row[1])
#     country_data.append(tempDict)  

In [113]:
language_data[:3]

[{'gender': 'Man', 'language': 'Assembly;Bash/Shell/PowerShell', 'count': 1},
 {'gender': 'Man',
  'language': 'Assembly;Bash/Shell/PowerShell;C#;Go;HTML/CSS;JavaScript;PHP;Python;Ruby;SQL;TypeScript;VBA',
  'count': 1},
 {'gender': 'Man',
  'language': 'Assembly;Bash/Shell/PowerShell;C#;Go;HTML/CSS;JavaScript;PHP;Swift;TypeScript',
  'count': 1}]

In [174]:
languages_by_gender

[{'man': {'Assembly': 648,
   'Bash/Shell/PowerShell': 3528,
   'C#': 3061,
   'Go': 816,
   'HTML/CSS': 6014,
   'JavaScript': 6419,
   'PHP': 2489,
   'Python': 3971,
   'Ruby': 749,
   'SQL': 5326,
   'TypeScript': 2060,
   'VBA': 533,
   'Swift': 586,
   'Java': 3921,
   'R': 564,
   'Scala': 383,
   'Objective-C': 430,
   'Rust': 303,
   'WebAssembly': 113,
   'Other(s):': 865,
   'C++': 2317,
   'Clojure': 139,
   'Dart': 168,
   'Elixir': 139,
   'Erlang': 86,
   'F#': 123,
   'Kotlin': 591,
   'C': 1995}},
 {'woman': {'HTML/CSS': 530,
   'JavaScript': 517,
   'Python': 313,
   'C#': 180,
   'SQL': 386,
   'Java': 284,
   'PHP': 182,
   'TypeScript': 130,
   'Bash/Shell/PowerShell': 225,
   'R': 75,
   'Go': 38,
   'Ruby': 85,
   'C': 121,
   'C++': 144,
   'Kotlin': 30,
   'Assembly': 55,
   'Objective-C': 25,
   'Swift': 44,
   'Other(s):': 58,
   'WebAssembly': 8,
   'Rust': 13,
   'VBA': 38,
   'Clojure': 3,
   'Dart': 6,
   'Erlang': 3,
   'Elixir': 10,
   'Scala': 26,
   '

In [171]:
languages_gender = []

In [172]:
for dic in languages_by_gender:
#     tempDict = {}
    for gender, languages in dic.items():
#         tempDict = {}
        
        for language, count in languages.items():
            tempDict = {}
            tempDict['gender'] = gender
            tempDict['language'] = language
            tempDict['count'] = int(count)
            
    languages_gender.append(tempDict)            
        

In [173]:
languages_gender

[{'gender': 'man', 'language': 'C', 'count': 1995},
 {'gender': 'woman', 'language': 'F#', 'count': 4},
 {'gender': 'other', 'language': 'Kotlin', 'count': 2}]

In [93]:
cur.execute('''
            SELECT Gender, LanguageWorkedWith, COUNT(LanguageWorkedWith)
            FROM jso11k WHERE LanguageWorkedWith > 0
            GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith)
        ''')

In [94]:
rows = cur.fetchall()

In [100]:
rows[:3]

[('Man',
  'Assembly;Bash/Shell/PowerShell;C#;Go;HTML/CSS;JavaScript;PHP;Python;Ruby;SQL;TypeScript;VBA',
  1),
 ('Man',
  'Assembly;Bash/Shell/PowerShell;C#;Go;HTML/CSS;JavaScript;PHP;Swift;TypeScript',
  1),
 ('Man', 'Assembly;Bash/Shell/PowerShell;C#;HTML/CSS', 1)]

In [96]:
languages_by_gender

[{'gender': 'man',
  'language_freq': {'Assembly': 648,
   'Bash/Shell/PowerShell': 3528,
   'C#': 3061,
   'Go': 816,
   'HTML/CSS': 6014,
   'JavaScript': 6419,
   'PHP': 2489,
   'Python': 3971,
   'Ruby': 749,
   'SQL': 5326,
   'TypeScript': 2060,
   'VBA': 533,
   'Swift': 586,
   'Java': 3921,
   'R': 564,
   'Scala': 383,
   'Objective-C': 430,
   'Rust': 303,
   'WebAssembly': 113,
   'Other(s):': 865,
   'C++': 2317,
   'Clojure': 139,
   'Dart': 168,
   'Elixir': 139,
   'Erlang': 86,
   'F#': 123,
   'Kotlin': 591,
   'C': 1995}},
 {'gender': 'woman',
  'language_freq': {'HTML/CSS': 530,
   'JavaScript': 517,
   'Python': 313,
   'C#': 180,
   'SQL': 386,
   'Java': 284,
   'PHP': 182,
   'TypeScript': 130,
   'Bash/Shell/PowerShell': 225,
   'R': 75,
   'Go': 38,
   'Ruby': 85,
   'C': 121,
   'C++': 144,
   'Kotlin': 30,
   'Assembly': 55,
   'Objective-C': 25,
   'Swift': 44,
   'Other(s):': 58,
   'WebAssembly': 8,
   'Rust': 13,
   'VBA': 38,
   'Clojure': 3,
   'Dart'

In [97]:
languages_by_gender = []

In [96]:
genders = ["Man", "Woman", "NOT IN ('Man', 'Woman')"]
db_column = "LanguageWorkedWith"

qery_obj = {}

for gender in genders:
    qery_obj[gender] = f'''
        SELECT {db_column}, COUNT({db_column})
        FROM jso11k WHERE Gender = {gender} and {db_column} > 0
        GROUP BY {db_column} ORDER BY COUNT({db_column}) DESC
    '''




In [97]:
qery_obj

{'Man': '\n        SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith)\n        FROM jso11k WHERE Gender = Man and LanguageWorkedWith > 0\n        GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith) DESC\n    ',
 'Woman': '\n        SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith)\n        FROM jso11k WHERE Gender = Woman and LanguageWorkedWith > 0\n        GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith) DESC\n    ',
 "NOT IN ('Man', 'Woman')": "\n        SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith)\n        FROM jso11k WHERE Gender = NOT IN ('Man', 'Woman') and LanguageWorkedWith > 0\n        GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith) DESC\n    "}

In [102]:
genders = ["= Man", " = Woman", "NOT IN ('Man', 'Woman')"]
# g = ['man', 'woman', 'other']
db_column = "LanguageWorkedWith"

qery_obj = {}

for gender in genders:
    qery_obj[gender] = (
    f"SELECT {db_column}, COUNT({db_column}) "
    f"FROM jso11k WHERE Gender {gender} and {db_column} > 0 "
    f"GROUP BY {db_column} ORDER BY COUNT({db_column}) DESC"
)

qery_obj

{'= Man': 'SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith) FROM jso11k WHERE Gender = Man and LanguageWorkedWith > 0 GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith) DESC',
 ' = Woman': 'SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith) FROM jso11k WHERE Gender  = Woman and LanguageWorkedWith > 0 GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith) DESC',
 "NOT IN ('Man', 'Woman')": "SELECT LanguageWorkedWith, COUNT(LanguageWorkedWith) FROM jso11k WHERE Gender NOT IN ('Man', 'Woman') and LanguageWorkedWith > 0 GROUP BY LanguageWorkedWith ORDER BY COUNT(LanguageWorkedWith) DESC"}

In [ ]:
app = Flask(__name__)

@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")

@app.route("/columns")
def names():
    """Return Column names from  a random sample  n = 11000 rows from the original dataset."""

    conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
    cur = conn.cursor()
    cur.execute("PRAGMA table_info(jso11k)")
    rows = cur.fetchall()
    column_names = [i[1] for i in rows]
    return jsonify(column_names)

    # COUNTRIES API
@app.route("/countries")
def countries():
    """
     Return a list of number of survey respondents by country
    """
    conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
    cur = conn.cursor()

    query_string = '''
        SELECT Country, count(country) FROM jso11k
        GROUP BY Country
        ORDER BY COUNT(Country) DESC
    '''

    cur.execute(query_string)
    rows = cur.fetchall()

    country_data = []

    for row in rows:
        tempDict = {}
        tempDict["country"] = row[0]
        tempDict["respondentCount"] = int(row[1])
        # tempDict[row[0]] = int(row[1])
        country_data.append(tempDict)

    return jsonify(country_data)

####################################################
####################################################

@app.route("/genders")
def genders():
    """
     Return a list of respondents count and percentages out of those who answered the Gender question
    """
    conn = sqlite3.connect("dash_app/db/js_overload.sqlite")
    cur = conn.cursor()

    genders_query = "SELECT Gender, COUNT(Gender) FROM jso11k GROUP BY Gender"

    cur.execute(genders_query)
    rows = cur.fetchall()

    gender_data = []
    for row in rows:
        temp_dict = {}
        temp_dict["gender"] = row[0]
        temp_dict["gender_count"] = row[1]
        gender_data.append(temp_dict)

    return jsonify(gender_data)
####################################################
####################################################

    # MAP CHART ROUTE
    # @TODO route for the map chart
# THE CSS STYLE IS JUST A DEMO
@app.route("/mapChart")
def mapChart():
    """Return the mapChart page."""
    return render_template("mapChart.html")
###################################################
####################################################
    # LINE CHART ROUTE
    # @TODO route for the LINE chart
# THE CSS STYLE IS JUST A DEMO
@app.route("/lineChart")
def lineChart():
    """Return the lineChart page."""
    return render_template("lineChart.html")

####################################################
####################################################
    # LINE CHART ROUTE
    # @TODO route for the donutChart chart
# THE CSS STYLE IS JUST A DEMO
@app.route("/donutChart")
def donutChart():
    """Return the lineChart page."""
    return render_template("donutChart.html")
    
####################################################
####################################################

    # BAR CHART ROUTE
    # @TODO route for the  BAR chart
# THE CSS STYLE IS JUST A DEMO
@app.route("/barChart")
def barChart():
    """Return the barChart page."""
    return render_template("barChart.html")####################################################

if __name__ == "__main__":
    app.run()
